In [1]:
import glob
import pandas as pd
from collections import defaultdict
from itertools import chain, combinations

# Укажите путь к вашим CSV-файлам (например, если они находятся в папке "data")
#all_files = glob.glob(r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\GeneratedLabelledFlows\TrafficLabelling/*.csv')

# Читаем все файлы в список DataFrame
#df_list = [pd.read_csv(file, encoding='cp1252', low_memory=False) for file in all_files]


# Объединяем все DataFrame в один общий DataFrame
#combined_df = pd.concat(df_list, ignore_index=True)

# Удаляем лишние пробелы в названиях столбцов
#combined_df.columns = combined_df.columns.str.strip()

# Теперь combined_df можно использовать для предобработки

def preprocess_data(df):
    """
    Преобразует DataFrame в список транзакций.
    Каждая транзакция – это список строк вида "ИмяСтолбца:Значение".
    Используются столбцы: Timestamp, Source IP, Source Port, Destination IP, 
    Destination Port, Protocol, Label.
    """
    transactions = []
    selected_columns = [ 'Total Fwd Packets','Flow Duration', 'Destination Port', 'Protocol', 'Label']
    for index, row in df.iterrows():
        transaction = []
        for col in selected_columns:
            value = str(row[col])
            item = f"{col}:{value}"
            transaction.append(item)
        transactions.append(transaction)
    return transactions

# Используем объединённый DataFrame для формирования транзакций
#transactions = preprocess_data(combined_df)

In [2]:
# Класс узла FP-дерева
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item          # Например, "Protocol:TCP"
        self.count = count        # Подсчет вхождений
        self.parent = parent      # Родительский узел
        self.children = {}        # Дочерние узлы: {элемент: FPNode}
        self.node_link = None     # Ссылка на следующий узел с таким же элементом

    def increment(self, count):
        self.count += count

def update_tree(items, tree, header_table, count):
    """
    Рекурсивно добавляет отсортированный список элементов в FP-дерево.
    Если элемент уже присутствует – увеличивает его счётчик, иначе создаёт новый узел.
    """
    first_item = items[0]
    if first_item in tree.children:
        tree.children[first_item].increment(count)
    else:
        new_node = FPNode(first_item, count, tree)
        tree.children[first_item] = new_node
        # Обновляем заголовочную таблицу: если узел отсутствует, записываем его,
        # иначе добавляем в цепочку через node_link.
        if header_table[first_item][1] is None:
            header_table[first_item][1] = new_node
        else:
            current = header_table[first_item][1]
            while current.node_link is not None:
                current = current.node_link
            current.node_link = new_node
    if len(items) > 1:
        update_tree(items[1:], tree.children[first_item], header_table, count)

def create_fp_tree(transactions, min_support):
    """
    Создает FP-дерево из транзакций:
      1. Подсчитывает поддержку каждого элемента.
      2. Отбрасывает элементы с поддержкой ниже min_support.
      3. Формирует заголовочную таблицу: {элемент: [поддержка, ссылка на первый узел]}.
      4. Для каждой транзакции оставляет только частые элементы, сортирует их по убыванию поддержки 
         и рекурсивно добавляет в дерево.
    """
    freq = {}
    for transaction in transactions:
        for item in transaction:
            freq[item] = freq.get(item, 0) + 1
    freq = {item: count for item, count in freq.items() if count >= min_support}
    if not freq:
        return None, None
    header_table = {item: [count, None] for item, count in freq.items()}
    root = FPNode('Null', 1, None)
    for transaction in transactions:
        transaction_items = [item for item in transaction if item in freq]
        if transaction_items:
            sorted_items = sorted(transaction_items, key=lambda item: header_table[item][0], reverse=True)
            update_tree(sorted_items, root, header_table, 1)
    return root, header_table

def ascend_fp_tree(node):
    """
    Поднимается по дереву от узла до корня (не включая корень)
    и возвращает список элементов пути.
    """
    path = []
    while node.parent is not None and node.parent.item != 'Null':
        node = node.parent
        path.append(node.item)
    return path

def find_prefix_paths(base_item, header_table):
    """
    Находит условную базу (prefix pattern base) для base_item.
    Возвращает словарь: {frozenset(пути): count}
    """
    conditional_patterns = {}
    node = header_table[base_item][1]
    while node:
        prefix_path = ascend_fp_tree(node)
        if prefix_path:
            conditional_patterns[frozenset(prefix_path)] = node.count
        node = node.node_link
    return conditional_patterns

def mine_fp_tree(tree, header_table, min_support, pre_fix, frequent_itemsets):
    """
    Рекурсивно добывает частые наборы элементов из FP-дерева.
    """
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])
    for base_item, base_info in sorted_items:
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_item)
        frequent_itemsets[frozenset(new_freq_set)] = base_info[0]
        conditional_patterns = find_prefix_paths(base_item, header_table)
        conditional_transactions = []
        for path, count in conditional_patterns.items():
            for _ in range(count):
                conditional_transactions.append(list(path))
        if conditional_transactions:
            conditional_tree, conditional_header = create_fp_tree(conditional_transactions, min_support)
            if conditional_header is not None:
                mine_fp_tree(conditional_tree, conditional_header, min_support, new_freq_set, frequent_itemsets)


In [3]:
def generate_association_rules(frequent_itemsets, min_conf):
    """
    Для каждого частого набора генерирует правила A -> B, 
    но оставляет только те, в которых B состоит исключительно из меток (Label:...).
    Рассчитывает достоверность: conf = support(A ∪ B) / support(A)
    Если conf >= min_conf, правило добавляется.
    """
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) < 2:
            continue
        # Перебираем все непустые подмножества itemset
        for subset in chain.from_iterable(combinations(itemset, r) for r in range(1, len(itemset))):
            A = set(subset)
            B = set(itemset) - A
            # Ограничиваем правила: оставляем только те, где все элементы B начинаются с "Label:"
            if B and all(item.startswith("Label:") for item in B) and frozenset(A) in frequent_itemsets:
                conf = frequent_itemsets[frozenset(itemset)] / frequent_itemsets[frozenset(A)]
                if conf >= min_conf:
                    rules.append((A, B, conf, frequent_itemsets[frozenset(itemset)]))
    return rules


def improved_fp_growth(transactions, min_sup, min_conf):
    """
    Улучшенный FP-growth объединяет:
      - Построение FP-дерева.
      - Добычу частых наборов (mine_fp_tree).
      - Генерацию ассоциативных правил.
    Возвращает frequent_itemsets и правила.
    """
    fp_tree, header_table = create_fp_tree(transactions, min_sup)
    frequent_itemsets = {}
    if fp_tree:
        mine_fp_tree(fp_tree, header_table, min_sup, set(), frequent_itemsets)
    rules = generate_association_rules(frequent_itemsets, min_conf)
    return frequent_itemsets, rules


In [4]:
# Для COFI‑дерева создаём отдельные классы и функции, чтобы не было конфликта с FP-деревом.

class COFINode:
    def __init__(self, item, count, parent):
        self.item = item                # Например, "Protocol:6"
        self.count = count              # Подсчет вхождений
        self.parent = parent            # Родительский узел
        self.children = {}              # Дочерние узлы: {элемент: COFINode}
        self.node_link = None           # Ссылка на следующий узел с таким же элементом
        self.participation_counter = 0  # Дополнительный счетчик (при необходимости)

    def increment(self, count):
        self.count += count

def update_cofi_tree(items, tree, header_table, count):
    first_item = items[0]
    if first_item in tree.children:
        tree.children[first_item].increment(count)
    else:
        new_node = COFINode(first_item, count, tree)
        tree.children[first_item] = new_node
        if header_table[first_item][1] is None:
            header_table[first_item][1] = new_node
        else:
            current = header_table[first_item][1]
            while current.node_link:
                current = current.node_link
            current.node_link = new_node
    if len(items) > 1:
        update_cofi_tree(items[1:], tree.children[first_item], header_table, count)

def create_cofi_tree_cofi(transactions, min_support):
    """
    Создает COFI‑дерево из транзакций (отдельная функция для COFI‑дерева).
    """
    freq = {}
    for transaction in transactions:
        for item in transaction:
            freq[item] = freq.get(item, 0) + 1
    freq = {item: count for item, count in freq.items() if count >= min_support}
    if not freq:
        return None, None
    header_table = {item: [count, None] for item, count in freq.items()}
    root = COFINode('Null', 1, None)
    for transaction in transactions:
        transaction_items = [item for item in transaction if item in freq]
        if transaction_items:
            sorted_items = sorted(transaction_items, key=lambda item: header_table[item][0], reverse=True)
            update_cofi_tree(sorted_items, root, header_table, 1)
    return root, header_table

def build_cofi_tree_for_item(base_item, fp_header, min_support):
    """
    Извлекает условную базу (prefix pattern base) для base_item из FP-дерева
    и строит COFI‑дерево для этих транзакций.
    """
    conditional_patterns = find_prefix_paths(base_item, fp_header)
    cofi_transactions = []
    for path, count in conditional_patterns.items():
        for _ in range(count):
            cofi_transactions.append(list(path))
    return create_cofi_tree_cofi(cofi_transactions, min_support)

def mine_cofi_tree(tree, header_table, min_support, pre_fix, cofi_patterns):
    """
    Рекурсивно добывает частые наборы элементов из COFI‑дерева.
    """
    sorted_items = sorted(header_table.items(), key=lambda x: x[1][0])
    for base_item, base_info in sorted_items:
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_item)
        cofi_patterns[frozenset(new_freq_set)] = base_info[0]
        conditional_patterns = find_prefix_paths(base_item, header_table)
        conditional_transactions = []
        for path, count in conditional_patterns.items():
            for _ in range(count):
                conditional_transactions.append(list(path))
        if conditional_transactions:
            conditional_tree, conditional_header = create_cofi_tree_cofi(conditional_transactions, min_support)
            if conditional_header is not None:
                mine_cofi_tree(conditional_tree, conditional_header, min_support, new_freq_set, cofi_patterns)

def build_sd_structure(cofi_patterns, min_support):
    """
    Группирует наборы из COFI‑дерева по длине и корректирует их поддержку.
    Для каждого набора меньшей длины ищутся наборы с большей длиной, содержащие его,
    и поддержка увеличивается.
    """
    sd_segments = {}
    for pattern, support in cofi_patterns.items():
        length = len(pattern)
        sd_segments.setdefault(length, []).append((pattern, support))
    
    updated_patterns = {}
    for length in sorted(sd_segments.keys()):
        for pattern, support in sd_segments[length]:
            updated_support = support
            for longer_length in sorted(sd_segments.keys()):
                if longer_length > length:
                    for longer_pattern, longer_support in sd_segments[longer_length]:
                        if pattern.issubset(longer_pattern):
                            updated_support += longer_support
            updated_patterns[pattern] = updated_support
    
    final_patterns = {pattern: sup for pattern, sup in updated_patterns.items() if sup >= min_support}
    return final_patterns


In [5]:
# Загрузка данных CICIDS 2017 и предобработка
#transactions = preprocess_data(combined_df)

df = pd.read_csv(r'C:\Users\Гребенников Матвей\Desktop\Диплом\Диплом\GeneratedLabelledFlows\TrafficLabelling\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
df.columns = df.columns.str.strip()
transactions = preprocess_data(df)

# Задаем параметры: минимальный порог поддержки и достоверности
min_support = 10
min_conf = 0.8

# 1. Построение FP-дерева (MDFP) и добыча частых наборов
fp_tree, header_table = create_fp_tree(transactions, min_support)
frequent_itemsets = {}
if fp_tree is None:
    print("Нет частых элементов, удовлетворяющих min_support.")
else:
    mine_fp_tree(fp_tree, header_table, min_support, set(), frequent_itemsets)
    print("Частые наборы (MDFP):")
    for itemset, support in sorted(frequent_itemsets.items(), key=lambda x: x[1], reverse=True):
        print(set(itemset), "поддержка:", support)

    # 2. Улучшенный FP-growth (объединяет этапы из 4.3)
    freq_itemsets_improved, association_rules = improved_fp_growth(transactions, min_support, min_conf)
    print("\nЧастые наборы (улучшенный FP-growth):")
    for itemset, support in sorted(freq_itemsets_improved.items(), key=lambda x: x[1], reverse=True):
        print(set(itemset), "поддержка:", support)
        
    print("\nАссоциативные правила (min_conf = {}):".format(min_conf))
    for A, B, conf, sup in association_rules:
        print(f"{A} -> {B} (conf: {conf:.2f}, support: {sup})")
    

 # 3. Применение COFI-дерева и структуры SD:
    # Используем несколько базовых элементов 
    base_items = [
    "Label:BENIGN",
    "Label:DDoS",            # если есть такие метки
    "Label:PortScan",       # если есть такие метки
    "Protocol:6",           # TCP
    "Protocol:17",          # UDP
    "Destination Port:80",
    "Destination Port:443",
    "Destination Port:53",
    "Protocol:6",
    "Protocol:17",
    "Protocol:1"
                 ]
    all_cofi_patterns = {}
    for base_item in base_items:
        if base_item in header_table:
            # Строим COFI-дерево по транзакциям, где содержится base_item
            cofi_tree, cofi_header = create_cofi_tree_cofi([t for t in transactions if base_item in t], min_support)
            if cofi_tree and cofi_header:
                cofi_patterns = {}
                mine_cofi_tree(cofi_tree, cofi_header, min_support, pre_fix=set([base_item]), cofi_patterns=cofi_patterns)
                print(f"\nЧастые наборы из COFI-дерева для {base_item}:")
                for itemset, support in sorted(cofi_patterns.items(), key=lambda x: x[1], reverse=True):
                    print(set(itemset), "поддержка:", support)
                all_cofi_patterns.update(cofi_patterns)
            else:
                print(f"\nНет условных транзакций для {base_item} с min_support = {min_support}.")
        else:
            print(f"\nЭлемент {base_item} не найден в FP-дереве.")
    
    print("\n--- Применение структуры SD ко всем полученным наборам ---")
    sd_patterns = build_sd_structure(all_cofi_patterns, min_support)
    for pattern, support in sorted(sd_patterns.items(), key=lambda x: x[1], reverse=True):
        print(set(pattern), "поддержка:", support) 




Частые наборы (MDFP):
{'Protocol:6'} поддержка: 192820
{'Destination Port:80'} поддержка: 136951
{'Protocol:6', 'Destination Port:80'} поддержка: 136951
{'Label:DDoS'} поддержка: 128027
{'Protocol:6', 'Label:DDoS'} поддержка: 128027
{'Destination Port:80', 'Label:DDoS'} поддержка: 128024
{'Protocol:6', 'Destination Port:80', 'Label:DDoS'} поддержка: 128024
{'Label:BENIGN'} поддержка: 97718
{'Protocol:6', 'Label:BENIGN'} поддержка: 64793
{'Total Fwd Packets:3'} поддержка: 61938
{'Protocol:6', 'Total Fwd Packets:3'} поддержка: 61883
{'Total Fwd Packets:3', 'Destination Port:80'} поддержка: 59579
{'Protocol:6', 'Total Fwd Packets:3', 'Destination Port:80'} поддержка: 59579
{'Total Fwd Packets:3', 'Label:DDoS'} поддержка: 56728
{'Protocol:6', 'Total Fwd Packets:3', 'Label:DDoS'} поддержка: 56728
{'Destination Port:80', 'Total Fwd Packets:3', 'Label:DDoS'} поддержка: 56728
{'Protocol:6', 'Destination Port:80', 'Total Fwd Packets:3', 'Label:DDoS'} поддержка: 56728
{'Protocol:17'} поддержка: 